In [379]:
import pandas as pd
import plotly.express as px
import re
import datetime
import numpy as np

from Project.Database import Db

# Example data for the TPM

In [380]:
# pd.read_csv("/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/Data/ISID.csv")

In [381]:
# pd.read_json(
#     path_or_buf="/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/output/Experiment_minsup0.1_minconf_0.1/level1.json",
#     orient='name_node')

In [382]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)

In [383]:
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self.index.str.contains("Child")) &
        (~self.index.str.contains("Prnt")) &
        (~self.index.str.contains("StatusLatentload")) &
        (~self.index.str.contains("Load_StatusClothesWasher")) &
        (~self.index.str.contains("Load_StatusDryerPowerTotal"))
))

appliances_statusOnOff = meta.loc[appliance_condition].index.tolist()

In [384]:
#Replaces values to be binary 0 or 1.
for i in appliances_statusOnOff:
    year1_minutes.loc[year1_minutes[i] < 0, i] = 0
    year1_minutes.loc[year1_minutes[i] > 0, i] = 1

In [385]:
minute_appliances_status = year1_minutes[["Timestamp"] + appliances_statusOnOff].copy()

In [386]:
for col in meta["Measurement_Location"].unique():
    appliance_condition = (lambda self: (
            (self["Parameter"] == "Status_OnOff") &
            (self.index.str.contains("Light")) &
            (self["Measurement_Location"] == col)
    ))
    light_cols = meta.loc[appliance_condition].index
    if len(light_cols) > 1:
        minute_appliances_status[col + " Lights"] = minute_appliances_status[light_cols].max(1)
        minute_appliances_status.drop(light_cols, axis= 1, inplace=True)
        for attribute in light_cols:
            appliances_statusOnOff.remove(attribute)
        appliances_statusOnOff.append(col + " Lights")

In [387]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
minute_appliances_status["Timestamp"] = pd.to_datetime(minute_appliances_status["Timestamp"],
                                                       format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(
    unit="h", arg=(
        minute_appliances_status["Timestamp"].astype("str").str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[
            1]).astype(
        int))

#Extracts hour of Timestamp.
minute_appliances_status["DayOfYear"] = minute_appliances_status.Timestamp.dt.dayofyear

In [388]:
minute_appliances_status["DayOfYear"] = minute_appliances_status.Timestamp.dt.dayofyear
minute_appliances_status["DayOfYear"] = minute_appliances_status["DayOfYear"] -  minute_appliances_status["DayOfYear"][0]
minute_appliances_status["DayOfYear"] = np.where(minute_appliances_status["DayOfYear"] < 0, minute_appliances_status["DayOfYear"] + 365, minute_appliances_status["DayOfYear"])
minute_appliances_status.loc[500000:].loc[lambda self: self["DayOfYear"] == 0] = 365
minute_appliances_status

/var/folders/bd/t85lw5j14_vc55x5tn1xbmr00000gn/T/ipykernel_2107/1015428806.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minute_appliances_status.loc[500000:].loc[lambda self: self["DayOfYear"] == 0] = 365


,Timestamp,Load_StatusApplianceCooktop,Load_StatusApplianceDishwasher,Load_StatusApplianceOven,Load_StatusApplianceRangeHood,Load_StatusBA1Lights,Load_StatusBA2Lights,Load_StatusBR2Lights,Load_StatusBR3Lights,Load_StatusBR4Lights,...,Load_StatusPlugLoadMBRTV,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadVacuum,Load_StatusPlugLoadVideoGame,Kitchen Lights,Living Room Lights,Master Bedroom Lights,DayOfYear
0,2013-06-30 23:01:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2013-06-30 23:02:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2013-06-30 23:03:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2013-06-30 23:04:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2013-06-30 23:05:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,2014-06-30 23:29:22+00:00,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365
518788,2014-06-30 23:30:22+00:00,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365
518789,2014-06-30 23:31:22+00:00,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365
518790,2014-06-30 23:59:22+00:00,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365


In [389]:
# minute_appliances_status['Timestamp'] = (pd.to_datetime(minute_appliances_status["Timestamp"],
#                                                        format="%Y-%m-%d %H:%M:%S%z", utc=True).astype("int") / 1e9).astype("int")

In [390]:
minute_appliances_status['Timestamp'] = (pd.DatetimeIndex ( minute_appliances_status["Timestamp"] ).astype ( np.int64 )/1e9).astype(int)


In [391]:
minute_appliances_status["DayOfYear"] = ((minute_appliances_status['Timestamp'] - minute_appliances_status['Timestamp'].min()) / (3600 * 24)).astype("int")

In [392]:
minute_appliances_status

,Timestamp,Load_StatusApplianceCooktop,Load_StatusApplianceDishwasher,Load_StatusApplianceOven,Load_StatusApplianceRangeHood,Load_StatusBA1Lights,Load_StatusBA2Lights,Load_StatusBR2Lights,Load_StatusBR3Lights,Load_StatusBR4Lights,...,Load_StatusPlugLoadMBRTV,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadVacuum,Load_StatusPlugLoadVideoGame,Kitchen Lights,Living Room Lights,Master Bedroom Lights,DayOfYear
0,1372633274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1372633334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1372633394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1372633454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1372633514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,1404170962,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365
518788,1404171022,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365
518789,1404171082,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365
518790,1404172762,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365


In [393]:
appliance_job_list = []

for appliance in appliances_statusOnOff:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load_Status")[-1]))
    appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][["Timestamp", "DayOfYear", appliance]][1:]

    df_indices = appliance_switch.index.tolist()
    for index, df_index in enumerate(df_indices):
        if appliance_switch.loc[df_index, appliance]:
            try:
                appliance_job_list.append({"start": appliance_switch.loc[df_index, "Timestamp"],
                                           "end": appliance_switch.loc[df_indices[index + 1], "Timestamp"],
                                           "appliance": name,
                                           "day": appliance_switch.loc[df_indices[index], "DayOfYear"]
                                           })
            except:
                continue

appliance_job_list

[{'start': 1372698031,
  'end': 1372699051,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': 1372780828,
  'end': 1372781848,
  'appliance': 'Appliance Cooktop',
  'day': 1},
 {'start': 1372863627,
  'end': 1372864647,
  'appliance': 'Appliance Cooktop',
  'day': 2},
 {'start': 1372946426,
  'end': 1372947446,
  'appliance': 'Appliance Cooktop',
  'day': 3},
 {'start': 1373029228,
  'end': 1373030248,
  'appliance': 'Appliance Cooktop',
  'day': 4},
 {'start': 1373076807,
  'end': 1373077767,
  'appliance': 'Appliance Cooktop',
  'day': 5},
 {'start': 1373090427,
  'end': 1373091447,
  'appliance': 'Appliance Cooktop',
  'day': 5},
 {'start': 1373115506,
  'end': 1373116526,
  'appliance': 'Appliance Cooktop',
  'day': 5},
 {'start': 1373159607,
  'end': 1373160627,
  'appliance': 'Appliance Cooktop',
  'day': 6},
 {'start': 1373173227,
  'end': 1373174247,
  'appliance': 'Appliance Cooktop',
  'day': 6},
 {'start': 1373194826,
  'end': 1373195846,
  'appliance': 'Appliance 

In [394]:
df = pd.DataFrame(appliance_job_list).sort_values(by=['day', 'start']).reset_index(drop=True)
df

,start,end,appliance,day
0,1372645757,1372645937,M B A Lights,0
1,1372647497,1372647677,M B A Lights,0
2,1372654697,1372655597,M B A Lights,0
3,1372654697,1372663397,Kitchen Lights,0
4,1372654997,1372655417,Plug Load Blender,0
...,...,...,...,...
18591,1404046751,1404052150,Plug Load M B R Blue Ray,363
18592,1404046751,1404052150,Plug Load M B R T V,363
18593,1404046751,1404050350,Master Bedroom Lights,363
18594,1404047051,1404048251,M B A Lights,363


In [395]:
df.to_csv(path_or_buf=Db.get_project_path("Project/04TPMAlgorithm/TPM/Data/houseDataWDryerAndWasher.csv"), header=False, index=False)

In [396]:
df1 = pd.read_csv(Db.get_project_path("Project/04TPMAlgorithm/TPM/Data/houseData.csv"), header=None)
df2 = pd.read_csv(Db.get_project_path("Project/04TPMAlgorithm/TPM/Data/houseDataWDryerAndWasher.csv"), header=None)

In [397]:
df1

,0,1,2,3
0,1372649357,1372649537,M B A Lights,0
1,1372651097,1372651277,M B A Lights,0
2,1372658297,1372659197,M B A Lights,0
3,1372658297,1372666997,Kitchen Lights,0
4,1372658597,1372659017,Plug Load Blender,0
...,...,...,...,...
18641,1404165562,1404172762,Plug Load Fan,364
18642,1404165562,1404166462,Plug Load Heating Pad,364
18643,1404165562,1404169162,Master Bedroom Lights,364
18644,1404165862,1404167062,M B A Lights,364


In [398]:
df2

,0,1,2,3
0,1372645757,1372645937,M B A Lights,0
1,1372647497,1372647677,M B A Lights,0
2,1372654697,1372655597,M B A Lights,0
3,1372654697,1372663397,Kitchen Lights,0
4,1372654997,1372655417,Plug Load Blender,0
...,...,...,...,...
18591,1404046751,1404052150,Plug Load M B R Blue Ray,363
18592,1404046751,1404052150,Plug Load M B R T V,363
18593,1404046751,1404050350,Master Bedroom Lights,363
18594,1404047051,1404048251,M B A Lights,363


In [399]:
df_diff = pd.concat([df1,df2]).drop_duplicates(keep=False, ignore_index=True)

In [400]:
df_diff

,0,1,2,3
0,1372649357,1372649537,M B A Lights,0
1,1372651097,1372651277,M B A Lights,0
2,1372658297,1372659197,M B A Lights,0
3,1372658297,1372666997,Kitchen Lights,0
4,1372658597,1372659017,Plug Load Blender,0
...,...,...,...,...
33749,1404046751,1404052150,Plug Load M B R Blue Ray,363
33750,1404046751,1404052150,Plug Load M B R T V,363
33751,1404046751,1404050350,Master Bedroom Lights,363
33752,1404047051,1404048251,M B A Lights,363
